In [2]:
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
from utilsCPD import *
import matplotlib.pyplot as plt
from SWCPD import BaseDetector as SWDCP
import os
from pathlib import Path

In [4]:
file_name = 'Occupancy.csv'

file_path = os.path.normpath(os.path.join(os.getcwd(),"datasets/Occupancy",file_name))

Occupancy = pd.read_csv(file_path).iloc[:,1:]

Targets = pd.read_csv(os.path.normpath(os.path.join(os.getcwd(),"datasets/Occupancy/OccupancyTargets.csv"))).iloc[:,1:]

GroundTruth = list(Targets.to_numpy().flatten())

Occupancy.head()

,Temperature,Humidity,Light,CO2
0,23.18,27.2720,426.0,721.25
1,23.15,27.2675,429.5,714.00
2,23.15,27.2450,426.0,713.50
3,23.15,27.2000,426.0,708.25
4,23.10,27.2000,426.0,704.50


### SWCPD

In [7]:
detector = SWDCP(data=Occupancy.to_numpy().astype(np.float32),window_length=500,max_history=500,significance=0.05)
print(detector.device)
detector.process_dataloader(p=2,n_theta=1000,seed=2025)
detector.evaluate(GroundTruth,30)

cpu


  5%|▍         | 346/7644 [00:03<01:16, 94.98it/s] 

Change detected at: 828 
Initiate new segment


 14%|█▍        | 1059/7644 [00:11<01:01, 106.40it/s]

Change detected at: 1544 
Initiate new segment


...

100%|█████████▉| 7618/7644 [01:15<00:00, 86.03it/s] 

Change detected at: 8101 
Initiate new segment


100%|██████████| 7644/7644 [01:16<00:00, 100.46it/s]

F1 score: 0.5833333333333334
Covering: 0.8027825975884822
0.5874125874125874


(0.5833333333333334,
 0.8027825975884822,
 0.5874125874125874,
 4,
 ({8101: 6,
   1544: 89,
   6655: 0,
   2988: 93,
   5232: 49,
   3731: 12,
   5787: 28,
   828: 3,
   7454: 239,
   2271: 0},
  51.9))

In [9]:
detector.evaluate(GroundTruth,30)

F1 score: 0.5833333333333334
Covering: 0.8027825975884822
0.5874125874125874


(0.5833333333333334,
 0.8027825975884822,
 0.5874125874125874,
 4,
 ({8101: 6,
   1544: 89,
   6655: 0,
   2988: 93,
   5232: 49,
   3731: 12,
   5787: 28,
   828: 3,
   7454: 239,
   2271: 0},
  51.9))

In [14]:
SWD_AUCS = [detector.evaluate(GroundTruth,tau)[2] for tau in [10,30,50,75,100]]
SWD_AUCS

F1 score: 0.41666666666666663
Covering: 0.8027825975884822
0.4195804195804196
F1 score: 0.5833333333333334
Covering: 0.8027825975884822
0.5874125874125874
F1 score: 0.6666666666666667
Covering: 0.8027825975884822
0.6713286713286712
F1 score: 0.6666666666666667
Covering: 0.8027825975884822
0.6713286713286712
F1 score: 0.8333333333333333
Covering: 0.8027825975884822
0.8391608391608393


[0.4195804195804196,
 0.5874125874125874,
 0.6713286713286712,
 0.6713286713286712,
 0.8391608391608393]

### Clasp

In [11]:
from claspy.segmentation import BinaryClaSPSegmentation

clasp = BinaryClaSPSegmentation(window_size=50).fit(Occupancy.to_numpy())
print('F1 & AUC & FP',f_measure({'0':GroundTruth},clasp.change_points,30))
covering({'0':GroundTruth},clasp.change_points,Occupancy.shape[0])
detection_delay(GroundTruth,clasp.change_points)

F1 & AUC & FP (0.2666666666666667, 0.5769230769230769, 0)


({828: 3}, 3.0)

In [12]:
CLASP_AUCS = [f_measure({'0':GroundTruth},clasp.change_points,tau)[2] for tau in [10,30,50,75,100]]
CLASP_AUCS

[0, 0, 0, 0, 0]

### Results for R methods

In [6]:
import json

#R=30,sig.lvl=0.05,min.size = 400
with open("./R/res/OccupancyECP.json") as f:
    res = json.load(f)[1:-1]

ECP_CPS =res
print("f1 & AUC & FP",f_measure({'0':GroundTruth},ECP_CPS,30))
print("Covering",covering({'0':GroundTruth},ECP_CPS,Occupancy.shape[0]))
print('Delay', detection_delay(GroundTruth,ECP_CPS)[1])

ECP_AUCS = [f_measure({'0':GroundTruth},ECP_CPS,tau)[1] for tau in [10,30,50,75,100]]
#BOCPD with datanormalization
with open("./R/res/OccupancyBOCPD.json") as f:
    res = json.load(f)[0][1:]

BOCPD_CPS =  res
print("F1 & AUC & FP",f_measure({'0':GroundTruth},BOCPD_CPS,30))
print("Covering",covering({'0':GroundTruth},BOCPD_CPS,Occupancy.shape[0]))
print('Delay',detection_delay(GroundTruth,BOCPD_CPS)[1])

BOCPD_AUCS = [f_measure({'0':GroundTruth},BOCPD_CPS,tau)[1] for tau in [10,30,50,75,100]]


print(ECP_AUCS)
print(BOCPD_AUCS)

f1 & AUC & FP (0.33333333333333337, 0.3393665158371041, 12)
Covering 0.6401316828686778
Delay 167.4375
F1 & AUC & FP (0.5454545454545455, 0.5711538461538461, 11)
Covering 0.7367589017005471
Delay 105.6842105263158
[0.2714932126696833, 0.3393665158371041, 0.40723981900452494, 0.6108597285067873, 0.6787330316742082]
[0.5711538461538461, 0.5711538461538461, 0.5711538461538461, 0.6346153846153847, 0.698076923076923]


## FP

### Results

| METHOD  | DD  | FP  |
|-------------|----------|-----------|
| SWD (ours)       | 52 | 4|
| ECP         | 167 | 12  |
| KCPA         |77 | 11|
| CLasP         | 3 | 0 |
| BOCPD         | 105  | 11 |